# Optimisations

Store all balls in a table so that i know what balls are nearby

and dont have to check all of them. To do this, I've made the radius constant

and the width of the table the diameter of the balls.

<br>

Going from looping over all the balls for every ball to only looping

over all of them around four times reduces the time from $ O(n^2) $ to $ O(4n) $.

Maybe.

In [44]:
# Define classes

class Vector2():
    x = None
    y = None
    
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y
    
    def __add__(self, other):
        if type(other) is Vector2:
            return Vector2(self.x + other.x, self.y + other.y)
        else:
            raise TypeError("Vector2 addition must be between vectors")
    
    def __radd__(self, other):
        if type(other) is Vector2:
            return Vector2(self.x + other.x, self.y + other.y)
        else:
            raise TypeError("Vector2 addition must be between vectors")
    
    def __sub__(self, other):
        if type(other) is Vector2:
            return Vector2(self.x - other.x, self.y - other.y)
        else:
            TypeError("Vector2 subtraction must be between vectors")

    def __rsub__(self, other):
        if type(other) is Vector2:
            return Vector2(other.x - self.x, other.y - self.y)
        else:
            TypeError("Vector2 subtraction must be between vectors")
    
    def __mul__(self, other):
        if type(other) is Vector2:
            # Scalar product
            return self.x * other.x + self.y * other.y
        elif type(other) is int or type(other) is float:
            # Vector scalar multiplication
            return Vector2(self.x * other, self.y * other)
        else:
            raise TypeError("Multiplication must be a number")
    
    def __rmul__(self, other):
        if type(other) is Vector2:
            # Scalar product
            return self.x * other.x + self.y * other.y
        elif type(other) is int or type(other) is float:
            # Vector scalar multiplication
            return Vector2(self.x * other, self.y * other)
        else:
            raise TypeError("Multiplication must be a number")
    
    def distance(self, other):
        if type(other) is Vector2:
            return ((self.x - other.x)**2 + (self.y - other.y)**2)**0.5
        else:
            raise TypeError("Can only find distance between vector2s")
    
    def get_magnitude(self):
        return ((self.x)**2 + (self.y)**2)**0.5
    
    def normalized(self):
        magnitude = self.get_magnitude()
        return Vector2(self.x / magnitude, self.y / magnitude)

    def __str__(self) -> str:
        return "[" + str(self.x) + ", " + str(self.y) + "]"

    def reflect(self, plane):
        # Credit for the formula goes to https://www.gamedev.net/forums/topic/165537-2d-vector-reflection/
        if not type(plane) is Vector2:
            raise TypeError("Reflection plane must be vector2")
        plane = plane.normalized()
        return (self - 2 * plane * (plane * self)) * -1


class Ball():
    position = None
    velocity = None
    radius = None
    overlapping = False
    collisions = []
    table_pos = None
    alone = False

    def __init__(self, pos, radius, velocity) -> None:
        self.position = pos
        self.radius = radius
        self.velocity = velocity

    def move_and_bounce(self, delta, rect_width, rect_height):
        nex_pos = self.position + self.velocity * delta
        if nex_pos.x + self.radius > rect_width or nex_pos.x - self.radius < 0:
            self.velocity.x *= -1
        if nex_pos.y + self.radius > rect_height or nex_pos.y - self.radius < 0:
            self.velocity.y *= -1
        self.position += self.velocity * delta
    
    def is_overlapping(self, objects):
        self.overlapping = False
        self.collisions = []
        for object in objects:
            if self.position.distance(object.position) < self.radius + object.radius and object != self:
                self.overlapping = True
                self.collisions.append(object)
        return self.overlapping

    def collide(self):
        for collision_object in self.collisions:
            collision_normal = (self.position - collision_object.position).normalized()

            new_self_vel = self.velocity.reflect(collision_normal) * -1
            # Only change velocity if the new velocity is pointing away from the collisionee.
            if new_self_vel * collision_normal > 0:
                self.velocity = new_self_vel

                new_obj_vel = collision_object.velocity.reflect(collision_normal * -1) * -1
                if new_obj_vel * (collision_normal * -1) > 0:
                    collision_object.velocity = new_obj_vel

In [118]:
from time import sleep, time
from random import randint
from ipycanvas import Canvas, hold_canvas
from math import floor

def get_surrounding_cells(table, row, column):
    cells = []
    for delta_row in range(-1, 2):
        if row + delta_row in table:
            for delta_column in range(-1, 2):
                if column + delta_column in table[row + delta_row]: # Then it is not the center cell
                    cells.append(table[row + delta_row][column + delta_column])
    return cells

canvas = Canvas(width=1200, height=700)
canvas.line_width = 3
canvas.fill_style = "red"
canvas.font = "20px arial"
display(canvas)

start_time = time()
previous_time = start_time
fps = 240
step = 0
spawning = True
ball_spawn_speed = 20
radius = 10
ball_speed = 40

# Initialize balls in a list
balls = []
for i in range(400):
    balls.append(Ball(Vector2(randint(radius, canvas.width - radius), randint(radius, canvas.height - radius)), # Position
                      radius,
                      Vector2(randint(-ball_speed, ball_speed), randint(-ball_speed, ball_speed)) # Velocity
    ))


# Main loop
while True: #time() - start_time < 40:
    # delta time buisness
    current_time = time()
    deltatime = current_time - previous_time
    previous_time = current_time

    # Add balls as time goes on
    # if spawning and len(balls) < 400 and step % ball_spawn_speed == 0:
    #     balls.append(Ball(Vector2(10, 10), # Position
    #                     radius,
    #                     Vector2(200, 300) # Velocity
    #     ))
    #     ball_spawn_speed =  max(ball_spawn_speed - 1, 2)


    #if deltatime < 0.076:
    #    spawning = False

    # Physics calculations

    # Im using a dict to avoid storing empty cells
    # This will be a table of lists, making it a 3D list
    table_of_positions = {}
    for ball in balls:
        row = floor(ball.position.y / (radius * 2))
        column = floor(ball.position.x / (radius * 2))

        if not row in table_of_positions:
            table_of_positions[row] = {column : [ball]}
        elif not column in table_of_positions[row]:
            table_of_positions[row][column] = [ball]
        else:
            table_of_positions[row][column].append(ball)


    # Overlapping detection
    for row in table_of_positions:
        for column in table_of_positions[row]:
            surrounding_cells = get_surrounding_cells(table_of_positions, row, column) # returns a 2D list
            objects = []
            for cell2 in surrounding_cells:
                for object in cell2:
                    objects.append(object)
            for ball in table_of_positions[row][column]:
                # Finally do the math
                ball.overlapping = False
                ball.collisions = []
                ball.alone = len(surrounding_cells) == 1
                for object in objects:
                    if ball.position.distance(object.position) < ball.radius + object.radius and object != ball:
                        ball.overlapping = True
                        ball.collisions.append(object)



    for ball in balls:
        ball.move_and_bounce(deltatime, canvas.width, canvas.height)
        ball.collide()

    with hold_canvas():
        canvas.clear()  # Draw after this
        # for i in range(0, canvas.width, radius*2):
        #     canvas.stroke_line(i, 0, i, canvas.width)
        #     canvas.stroke_line(0, i, canvas.width, i)

        for ball in balls:
            if ball.overlapping:
                canvas.fill_style = "red"
            elif ball.alone:
                canvas.fill_style = "yellow"
            else:
                canvas.fill_style = "orange"
            canvas.fill_circle(ball.position.x, ball.position.y, ball.radius)
            canvas.stroke_circle(ball.position.x, ball.position.y, ball.radius)
        
        canvas.fill_style = "black"
        canvas.fill_text("Balls: " + str(len(balls)), 10, 25)
        canvas.fill_text("Frames Per Second: " + str(round(1 / deltatime)), 10, 50)
        canvas.fill_text("Seconds Per Frame: " + str(round(deltatime, 3)), 10, 75)
    
    sleep(1/fps)
    step += 1

Canvas(height=700, width=1200)

KeyboardInterrupt: 